In [1]:
import numpy as np
import cv2

In [2]:
with_mask = np.load("with_mask.npy")
without_mask = np.load("without_mask.npy")

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask.shape

(200, 50, 50, 3)

In [5]:
with_mask = with_mask.reshape(200,50*50*3)#converting data into 2D
without_mask = without_mask.reshape(200,50*50*3)

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
x=np.r_[with_mask, without_mask]#r_ is used to concatinate data

In [9]:
x.shape

(400, 7500)

In [10]:
labels = np.zeros(x.shape[0])

In [11]:
labels[200:] = 1.0  # next 200 data images are with mask data
# if the output is zero we are wearing a mask or if output is 1 then we are not wearing a mask


In [12]:
names = {0:"Mask", 1:"No Mask"}

In [13]:
# ML algo
#svm- support vector machine
#svc- support vector classification
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [22]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=9137039b66a666b0733872894514180622fccf39f7fbbce638ec48bc91cc1958
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,labels,test_size=0.25)# means 25% data reserve for testing and 75% data for Ml


In [16]:
x_train.shape# as this data has 7500 colms, it will slow down the ML algo , for this we will use dimensinality reduction 
              #technique in ml
               

(300, 7500)

In [17]:
#x_train, x_test, y_train, y_test = train_test_split(x,labels,test_size=0.25)# for shuffling of data we again doing testing and
                                                                               #traing of data

In [18]:
# for minemsinality reduction 
#PCA- Principal component Analysis
from sklearn.decomposition import PCA

In [19]:
pca = PCA(n_components=3)# convert into 3 D
x_train = pca.fit_transform(x_train)

In [20]:
x_train.shape

(300, 3)

In [21]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [22]:
x_test = pca.fit_transform(x_test)
y_pred = svm.predict(x_test)

In [23]:
accuracy_score(y_test, y_pred)

0.98

In [24]:
haar_data = cv2.CascadeClassifier("data.xml")
capture = cv2.VideoCapture(0)# to open camera or if want to collect data from video then mention path of the video like(video.np4)
data = []
font = cv2.FONT_HERSHEY_COMPLEX

while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w, y+h),(255,0,255),4)
            face = img[y:y+h, x:x+w, :] 
            face = cv2.resize(face, (50,50)) 
            #face = pca.transform(face)
            face = face.reshape(-1, 3)
            pred = svm.predict(face)
            n = names[int(pred)][0]
            cv2.putText(img, n, (x,y), font, 1,(244,233,250), 2)
            print(n)
                
            
        cv2.imshow('musheer',img)
        if cv2.waitKey(2) == 27:   
            break
capture.release()
cv2.destroyAllWindows()

TypeError: only size-1 arrays can be converted to Python scalars